In [ ]:
import time
import sys
import os
import requests
import numpy as np
import subprocess
from scipy.signal import welch

# --- CONFIGURATION ---
DEVICE_NAME = "BA HALO"
API_URL = "http://localhost:8000/update-emotion"
WINDOW_SECONDS = 4
SAMPLING_RATE = 250

# --- AUTO-INSTALL DEPENDENCIES ---
# The BrainAccess library depends on these. We install them if missing.
required_packages = ["mne", "multimethod", "bleak", "bitstruct"]

for package in required_packages:
    try:
        __import__(package)
    except ImportError:
        print(f"Installing missing dependency: {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

import mne

# --- MOCK CLASSES FOR COLAB SIMULATION ---
# These are used if the hardware connection fails (likely in Colab)
class MockManager:
    def __enter__(self): return self
    def __exit__(self, exc_type, exc_val, exc_tb): pass
    def disconnect(self): pass

class MockEEG:
    def setup(self, manager, device_name, cap):
        print(f"[SIMULATION] Mock setup for {device_name} complete.")

    def start_acquisition(self):
        print("[SIMULATION] Started generating fake brainwaves...")
        self.start_time = time.time()

    def stop_acquisition(self):
        print("[SIMULATION] Stopped.")

    def get_mne(self, tim=4):
        # Generate fake data: 4 channels, 'tim' seconds long
        n_samples = int(tim * SAMPLING_RATE)
        t = np.linspace(0, tim, n_samples)

        # Generate random noise
        data = np.random.randn(4, n_samples) * 1e-6

        # Add artificial alpha wave (10Hz) to test detection logic
        alpha_wave = np.sin(2 * np.pi * 10 * t) * 20e-6
        data += alpha_wave

        info = mne.create_info(ch_names=['Fp1', 'Fp2', 'O1', 'O2'], sfreq=SAMPLING_RATE, ch_types='eeg')
        raw = mne.io.RawArray(data, info, verbose=False)
        return raw

# --- IMPORT HANDLING ---
USE_SIMULATION = False
try:
    # Ensure Python looks in the current directory for the 'brainaccess' folder
    if os.getcwd() not in sys.path:
        sys.path.append(os.getcwd())

    from brainaccess.utils import acquisition
    from brainaccess.core.eeg_manager import EEGManager
    print("Successfully imported BrainAccess library.")
except ImportError as e:
    print("\n" + "="*60)
    print("WARNING: BrainAccess library could not be loaded.")
    print(f"Reason: {e}")
    print("If you see 'No module named...', it means a dependency is still missing.")
    print("SWITCHING TO SIMULATION MODE.")
    print("="*60 + "\n")
    USE_SIMULATION = True
except Exception as e:
    # Catch other errors (like OS permission issues)
    print(f"\nWARNING: Error loading library: {e}")
    print("SWITCHING TO SIMULATION MODE.")
    USE_SIMULATION = True

def get_band_power(data_segment):
    """
    Manual calculation of Alpha/Beta power using SciPy
    """
    # Average across all channels
    avg_data = np.mean(data_segment, axis=0)

    # Compute Power Spectral Density
    freqs, psd = welch(avg_data, fs=SAMPLING_RATE, nperseg=SAMPLING_RATE)

    # Define bands
    alpha_band = (8, 13)
    beta_band = (13, 30)
    theta_band = (4, 8)

    # Extract power
    alpha = np.mean(psd[(freqs >= alpha_band[0]) & (freqs <= alpha_band[1])])
    beta = np.mean(psd[(freqs >= beta_band[0]) & (freqs <= beta_band[1])])
    theta = np.mean(psd[(freqs >= theta_band[0]) & (freqs <= theta_band[1])])

    return {"alpha": alpha, "beta": beta, "theta": theta}

def main():
    # 1. SETUP
    print(f"Searching for {DEVICE_NAME}...")

    if USE_SIMULATION:
        eeg = MockEEG()
        mgr_class = MockManager
    else:
        try:
            eeg = acquisition.EEG()
            mgr_class = EEGManager
        except Exception as e:
            print(f"Hardware init failed: {e}. Switching to simulation.")
            eeg = MockEEG()
            mgr_class = MockManager

    with mgr_class() as mgr:
        cap = {0: "Fp1", 1: "Fp2", 2: "O1", 3: "O2"}

        try:
            eeg.setup(mgr, device_name=DEVICE_NAME, cap=cap)
            eeg.start_acquisition()
            print("Stream started.")
        except Exception as e:
            print(f"Could not connect: {e}")
            return

        # 2. MAIN LOOP
        try:
            while True:
                time.sleep(1)

                raw = eeg.get_mne(tim=WINDOW_SECONDS)

                if raw is None or len(raw) == 0:
                    continue

                data = raw.get_data()

                if data.shape[1] < SAMPLING_RATE:
                    continue

                # 3. CALCULATE & CLASSIFY
                bands = get_band_power(data)

                emotion = "Neutral"
                ratio = bands["beta"] / (bands["alpha"] + 1e-9)

                if ratio > 1.2:
                    emotion = "Stressed"
                elif bands["theta"] > bands["alpha"]:
                    emotion = "Sad"
                elif bands["alpha"] > bands["beta"]:
                    emotion = "Calm"
                else:
                    emotion = "Happy"

                print(f"A: {bands['alpha']:.2e} | B: {bands['beta']:.2e} -> {emotion}")

                # 4. SEND TO API
                try:
                    # In simulation, we just print the payload intended for the API
                    # requests.post(API_URL, json={"label": emotion})
                    pass
                except:
                    pass

        except KeyboardInterrupt:
            print("Stopping...")
        finally:
            eeg.stop_acquisition()
            # Only call disconnect if it exists
            if hasattr(mgr, 'disconnect'):
                mgr.disconnect()

if __name__ == "__main__":
    main()

Successfully imported BrainAccess library.
Searching for BA HALO...
Hardware init failed: Library already initialized. Switching to simulation.
[SIMULATION] Mock setup for BA HALO complete.
[SIMULATION] Started generating fake brainwaves...
Stream started.
A: 3.33e-11 | B: 1.98e-15 -> Calm
A: 3.32e-11 | B: 1.72e-15 -> Calm
A: 3.35e-11 | B: 1.71e-15 -> Calm
A: 3.34e-11 | B: 2.14e-15 -> Calm
A: 3.34e-11 | B: 1.51e-15 -> Calm
A: 3.33e-11 | B: 1.73e-15 -> Calm
A: 3.33e-11 | B: 2.22e-15 -> Calm
A: 3.33e-11 | B: 2.04e-15 -> Calm
A: 3.33e-11 | B: 2.06e-15 -> Calm
A: 3.34e-11 | B: 1.87e-15 -> Calm
A: 3.33e-11 | B: 1.45e-15 -> Calm
A: 3.31e-11 | B: 2.12e-15 -> Calm
A: 3.33e-11 | B: 1.99e-15 -> Calm
A: 3.32e-11 | B: 2.32e-15 -> Calm
A: 3.34e-11 | B: 2.01e-15 -> Calm
Stopping...
[SIMULATION] Stopped.


In [13]:
import time
import sys
import os
import requests
import numpy as np
import torch
from scipy.signal import welch
import mne

# --- IMPORTS (STRICT) ---
try:
    from brainaccess.utils import acquisition
    from brainaccess.core.eeg_manager import EEGManager
    from emotion_model import AdvancedEmotionModel
except ImportError as e:
    print(f"CRITICAL ERROR: Missing dependency. {e}")
    sys.exit(1)

# --- CONFIGURATION ---
DEVICE_NAME = "BA HALO"
API_URL = "http://localhost:8000/update-emotion"
WINDOW_SECONDS = 4
SAMPLING_RATE = 250
MODEL_PATH = "advanced_eeg_model.pth"

# --- HELPER: DATA PROCESSOR ---
def process_window_for_nn(data, sampling_rate):
    """
    Splits 4s window into 8 chunks -> Returns Tensor for Model
    Returns None if signal is flat/zero.
    """
    # Check for dead signal (all zeros or NaNs)
    if np.all(data == 0) or np.isnan(data).any():
        return None

    # Split into 8 segments (0.5s each)
    n_splits = 8
    try:
        splits = np.array_split(data, n_splits, axis=1)
    except:
        return None

    sequence = []

    for segment in splits:
        avg_data = np.mean(segment, axis=0)
        # Handle empty segments
        if len(avg_data) == 0: return None

        freqs, psd = welch(avg_data, fs=sampling_rate, nperseg=len(avg_data))

        def get_power(low, high):
            mask = (freqs >= low) & (freqs <= high)
            if not np.any(mask): return 0.0
            return np.mean(psd[mask])

        alpha = get_power(8, 13)
        beta  = get_power(13, 30)
        theta = get_power(4, 8)

        sequence.append([alpha, beta, theta])

    # Return Shape: (1, 8, 3)
    return torch.tensor([sequence], dtype=torch.float32)

# --- HELPER: MAPPING ---
def map_to_label(valence, energy):
    if valence >= 0.5 and energy >= 0.5: return "Happy"
    elif valence < 0.5 and energy >= 0.5: return "Stressed"
    elif valence < 0.5 and energy < 0.5: return "Sad"
    else: return "Calm"

def send_to_backend(mood):
    try:
        requests.post(API_URL, json={"label": mood}, timeout=0.2)
    except:
        pass # Backend might be down, keep running

# --- MAIN EXECUTION ---
def main():
    # 1. LOAD MODEL
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Initializing Model on {device}...")

    model = AdvancedEmotionModel(input_features=3).to(device)

    if not os.path.exists(MODEL_PATH):
        print(f"ERROR: Model weights '{MODEL_PATH}' not found.")
        print("Please run the training/weight creation script first.")
        sys.exit(1)

    try:
        model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
        model.eval()
    except Exception as e:
        print(f"Model Load Error: {e}")
        sys.exit(1)

    # 2. HARDWARE CONNECTION
    print(f"Connecting to {DEVICE_NAME}...")
    eeg = acquisition.EEG()

    with EEGManager() as mgr:
        try:
            # Map channels (Ensure this matches your physical cap setup)
            cap = {0: "Fp1", 1: "Fp2", 2: "O1", 3: "O2"}
            eeg.setup(mgr, device_name=DEVICE_NAME, cap=cap)
            eeg.start_acquisition()
            print(">>> EEG STREAM STARTED <<<")
        except Exception as e:
            print(f"CONNECTION FAILED: {e}")
            print("Check bluetooth/USB and try again.")
            sys.exit(1)

        # 3. REAL-TIME LOOP
        try:
            while True:
                time.sleep(1) # Wait for buffer to fill

                # Fetch MNE Raw Object
                raw = eeg.get_mne(tim=WINDOW_SECONDS)

                # --- SAFETY CHECKS ---
                if raw is None or len(raw) == 0:
                    print("No Data Received -> Sending Neutral")
                    send_to_backend("Neutral")
                    continue

                data = raw.get_data()

                # Check dimensions (Must have enough samples)
                if data.shape[1] < (SAMPLING_RATE * 0.5):
                    continue

                # --- PROCESSING ---
                input_tensor = process_window_for_nn(data, SAMPLING_RATE)

                if input_tensor is None:
                    # Signal is flat/bad
                    print("Bad Signal/Noise -> Sending Neutral")
                    send_to_backend("Neutral")
                    continue

                # --- INFERENCE ---
                input_tensor = input_tensor.to(device)

                with torch.no_grad():
                    v_score, e_score = model(input_tensor)
                    v = v_score.item()
                    e = e_score.item()

                mood = map_to_label(v, e)
                print(f"Signal OK | V:{v:.2f} E:{e:.2f} -> {mood}")

                send_to_backend(mood)

        except KeyboardInterrupt:
            print("\nStopping Stream...")
        finally:
            eeg.stop_acquisition()
            # mgr context manager handles disconnect automatically

if __name__ == "__main__":
    main()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



CRITICAL ERROR: Missing dependency. No module named 'emotion_model'
Traceback (most recent call last):
  File "/tmp/ipython-input-1813969223.py", line 14, in <cell line: 0>
    from emotion_model import AdvancedEmotionModel
ModuleNotFoundError: No module named 'emotion_model'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-1813969223.py", line 17, in <cell line: 0>
    sys.exit(1)
SystemExit: 1

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

TypeError: object of type 'NoneType' has no len()

In [9]:
import time
import sys
import os
import requests
import numpy as np
import subprocess
from scipy.signal import welch
import joblib  # Needed for loading the model

# --- CONFIGURATION ---
DEVICE_NAME = "BA HALO"
API_URL = "http://localhost:8000/update-emotion"
WINDOW_SECONDS = 4
SAMPLING_RATE = 250
MODEL_PATH = "AdvancedEmotionModel"

# --- AUTO-INSTALL DEPENDENCIES ---
required_packages = ["mne", "multimethod", "bleak", "bitstruct", "scikit-learn", "pandas"]

for package in required_packages:
    try:
        __import__(package)
    except ImportError:
        # Handle scikit-learn import name difference
        pkg_name = "sklearn" if package == "scikit-learn" else package
        try:
            __import__(pkg_name)
        except ImportError:
            print(f"Installing missing dependency: {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])

import mne

# --- MOCK CLASSES (Unchanged) ---
class MockManager:
    def __enter__(self): return self
    def __exit__(self, exc_type, exc_val, exc_tb): pass
    def disconnect(self): pass

class MockEEG:
    def setup(self, manager, device_name, cap):
        print(f"[SIMULATION] Mock setup for {device_name} complete.")
    def start_acquisition(self):
        print("[SIMULATION] Started generating fake brainwaves...")
        self.start_time = time.time()
    def stop_acquisition(self):
        print("[SIMULATION] Stopped.")
    def get_mne(self, tim=4):
        n_samples = int(tim * SAMPLING_RATE)
        t = np.linspace(0, tim, n_samples)
        data = np.random.randn(4, n_samples) * 1e-6
        # Simulate 'Stressed' (High Beta) for testing
        beta_wave = np.sin(2 * np.pi * 20 * t) * 20e-6
        data += beta_wave
        info = mne.create_info(ch_names=['Fp1', 'Fp2', 'O1', 'O2'], sfreq=SAMPLING_RATE, ch_types='eeg')
        raw = mne.io.RawArray(data, info, verbose=False)
        return raw

# --- IMPORT HANDLING (Unchanged) ---
USE_SIMULATION = False
try:
    if os.getcwd() not in sys.path:
        sys.path.append(os.getcwd())
    from brainaccess.utils import acquisition
    from brainaccess.core.eeg_manager import EEGManager
    print("Successfully imported BrainAccess library.")
except ImportError as e:
    print(f"WARNING: BrainAccess library not found. Reason: {e}")
    USE_SIMULATION = True
except Exception as e:
    print(f"WARNING: Error loading library: {e}")
    USE_SIMULATION = True

# --- FEATURE EXTRACTION ---
def extract_features(data_segment):
    """
    Returns a 2D array of features expected by the model:
    [[Alpha, Beta, Theta, Delta, Gamma]]
    """
    # Average across all channels (Spatial averaging)
    avg_data = np.mean(data_segment, axis=0)

    # Compute PSD
    freqs, psd = welch(avg_data, fs=SAMPLING_RATE, nperseg=SAMPLING_RATE)

    # Define bands
    bands = {
        "Alpha": (8, 13),
        "Beta": (13, 30),
        "Theta": (4, 8),
        "Delta": (0.5, 4),
        "Gamma": (30, 50)
    }

    powers = []
    # ORDER MATTERS: Must match the order used in train_model.py
    # Order: Alpha, Beta, Theta, Delta, Gamma
    for band_name in ["Alpha", "Beta", "Theta", "Delta", "Gamma"]:
        low, high = bands[band_name]
        # Find intersecting frequencies
        idx = np.logical_and(freqs >= low, freqs <= high)
        power = np.mean(psd[idx])
        powers.append(power)

    # Normalize features (simple relative power approximation) to match training scale
    # In a real scenario, use StandardScaler from sklearn
    powers = np.array(powers)
    return powers.reshape(1, -1)

def main():
    # 1. SETUP MODEL
    try:
        clf = joblib.load(MODEL_PATH)
        print(f"Loaded ML Model from {MODEL_PATH}")
    except Exception as e:
        print("ERROR: Model not found. Please run 'train_model.py' first.")
        print("Falling back to dummy logic.")
        clf = None

    # 2. SETUP DEVICE
    print(f"Searching for {DEVICE_NAME}...")
    if USE_SIMULATION:
        eeg = MockEEG()
        mgr_class = MockManager
    else:
        try:
            eeg = acquisition.EEG()
            mgr_class = EEGManager
        except Exception as e:
            eeg = MockEEG()
            mgr_class = MockManager

    with mgr_class() as mgr:
        cap = {0: "Fp1", 1: "Fp2", 2: "O1", 3: "O2"}
        try:
            eeg.setup(mgr, device_name=DEVICE_NAME, cap=cap)
            eeg.start_acquisition()
            print("Stream started.")
        except Exception as e:
            print(f"Could not connect: {e}")
            return

        # 3. MAIN LOOP
        try:
            while True:
                time.sleep(1) # Wait for buffer to fill

                raw = eeg.get_mne(tim=WINDOW_SECONDS)
                if raw is None or len(raw) == 0: continue
                data = raw.get_data()
                if data.shape[1] < SAMPLING_RATE: continue

                # --- ML PREDICTION ---
                features = extract_features(data) # Returns shape (1, 5)

                if clf:
                    # Predict using the model
                    prediction = clf.predict(features)[0]

                    # Optional: Get confidence score
                    probs = clf.predict_proba(features)
                    confidence = np.max(probs)

                    print(f"Model sees: {prediction} (Conf: {confidence:.2f})")
                else:
                    prediction = "Neutral" # Fallback

                # --- SEND TO API ---
                # This assumes your frontend/backend has an endpoint to receive this
                try:
                    payload = {"mood": prediction}
                    print(f"Sending to Frontend: {payload}")

                    # Uncomment below to actually send
                    # requests.post(API_URL, json=payload, timeout=0.5)
                except Exception as e:
                    print(f"API Error: {e}")

        except KeyboardInterrupt:
            print("Stopping...")
        finally:
            eeg.stop_acquisition()
            if hasattr(mgr, 'disconnect'):
                mgr.disconnect()

if __name__ == "__main__":
    main()

Successfully imported BrainAccess library.
ERROR: Model not found. Please run 'train_model.py' first.
Falling back to dummy logic.
Searching for BA HALO...
[SIMULATION] Mock setup for BA HALO complete.
[SIMULATION] Started generating fake brainwaves...
Stream started.
Sending to Frontend: {'mood': 'Neutral'}
Sending to Frontend: {'mood': 'Neutral'}
Sending to Frontend: {'mood': 'Neutral'}
Sending to Frontend: {'mood': 'Neutral'}
Sending to Frontend: {'mood': 'Neutral'}
Sending to Frontend: {'mood': 'Neutral'}
Stopping...
[SIMULATION] Stopped.
